In [85]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar
from dateutil.relativedelta import relativedelta
import math
import re
import pyodbc



# Reading Source Data 'ADT_Dummy Data' and 'ADT_ER_IP_04012021to07062021'

server = 'wn000038725' 
database = 'oas_mas_stg' 
username = 'svcoimas'
password = 'Canopyanalytics#124'

#cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()


df = pd.read_excel(r'C:\Users\ssing296\Desktop\Canopy\MyCanopy Dashboard\Kinvey_Registration_Master_File_Data.xlsx')

#df['Diagnosis'].fillna('blank_code', inplace=True)

df.columns = df.columns.str.replace(' ', '')

#df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

#print(df['MemberIDofRegistrant'].str.isalpha())

#df['MemberIDofRegistrant'] = df['MemberIDofRegistrant'].str.strip()


#print(df['MemberIDofRegistrant'].dtype)


df = df.reset_index()
"""
for index, row in df.iterrows():
    if(isinstance(row['MemberIDofRegistrant'],int)):
        print("YES")
    else:
        row['MemberIDofRegistrant']=row['MemberIDofRegistrant'].strip()
        print(row['MemberIDofRegistrant'])
"""

def whitespace_remover(dataframe):
    for i in dataframe.columns:
        #print(dataframe[i].dtype)
        if dataframe[i].dtype == 'object':
            for index, row in df.iterrows():
                if(isinstance(row[i],int) or isinstance(row[i],float)):
                    row[i]
                else:
                    row[i] = row[i].strip()
        else:
            pass
whitespace_remover(df)

qry_insert = "Insert into  [Canopy].[Registration_Master_Test] ([Date of Registration],[Time of Registration],[Member ID of Registrant],[Kinvey ID of Registrant],[Member ID of Primary Subscriber],[Kinvey ID of subscriber],[Group ID of registrant],[Account Description],[Payor of Registrant],[IPA of Registrant],[Category Name of Registrant],[DOB of Registrant],[Age]) values(?,?,?,?,?,?,?,?,?,?,?,?,?)"

cursor.execute("truncate table [Canopy].[Registration_Master_Test]")

def check_null(column_value):
    
    if(pd.isnull(column_value)):
        column_value = None
    else:
        column_value
    return column_value

insert_batch = []
rec = ""
counter = 0;
batchnum = 1;

data_len = len(df)


for index,row in df.iterrows():

    rec = (check_null(row['DateofRegistration']),check_null(row['TimeofRegistration']),check_null(row['MemberIDofRegistrant']),check_null(row['KinveyIDofRegistrant']),check_null(row['MemberIDofPrimarySubscriber']),check_null(row['KinveyIDofsubscriber']),check_null(row['GroupIDofregistrant']),check_null(row['AccountDescription']),check_null(row['PayorofRegistrant']),check_null(row['IPAofRegistrant']),check_null(row['CategoryNameofRegistrant']),check_null(row['DOBofRegistrant']),check_null(row['Age'])) 
    
    insert_batch.append(rec)
    counter = counter + 1;
    if(counter%100 == 0):
        cursor.executemany(qry_insert,insert_batch)
        cursor.commit()
        counter = 0
        data_len = data_len - len(insert_batch)
        insert_batch = []
        batchnum = batchnum + 1

if data_len != 0:
    cursor.executemany(qry_insert,insert_batch)  

    cursor.execute(qry_insert,rec)
    
print("Data Load Done")
        
        
        


cnxn.commit()
cursor.close()

Data Load Done
